In [1]:
import requests
import json
import pandas as pd
import numpy as np
import bs4
import glob
import os
import time
import re

In [4]:
import fish

In [5]:
response = requests.get('https://stardewvalleywiki.com/Crafting')
crafting = bs4.BeautifulSoup(response.content, features='lxml')
url = 'https://stardewvalleywiki.com{}'

In [66]:
crafting_df = pd.DataFrame(columns = ['Name', 'Recipe Source', 'Ingredients'])
for i in tables:
    for k in i.find_all('td'):
        if len(k) == 2:
            link = k.find('a')['href']
            new_url = url.format(link)
            response = requests.get(new_url)
            page = bs4.BeautifulSoup(response.content, features='lxml')
            name = page.find('h1').text.strip()
            if page.find_all('tbody')[0].text.strip().split()[0] == 'Warning:':
                table = page.find_all('tbody')[1]
            else:
                table = page.find_all('tbody')[0]
            for j in table.find_all('tr'):
                category = j.find('td').text.strip().replace('(s)', '').replace('Recipe ', '')
                if category == 'Source:':
                    recipe_source = j.find_all('td')[1].text.strip()
                if category == 'Ingredients:':
                    ingredients = {}
                    for p in j.find_all('td')[1].find_all('span'):
                        lst = p.text.strip().split('\xa0')
                        lst[1] = int(lst[1].strip('()'))
                        ingredients[lst[0]] = lst[1]
                else:
                    continue
        
            result_dict = {'Name': name, 'Recipe Source': recipe_source, 'Ingredients': ingredients}
            crafting_df = crafting_df.append(result_dict, ignore_index=True)
        else:
            continue


In [68]:
crafting_df

,Name,Recipe Source,Ingredients
0,Wood Fence,Starter,{'Wood': 2}
1,Stone Fence,Farming (Level 2),{'Stone': 2}
2,Iron Fence,Farming (Level 4),{'Iron Bar': 1}
3,Hardwood Fence,Farming (Level 6),{'Hardwood': 1}
4,Grass Starter,"Pierre's for 1,000g",{'Fiber': 10}
...,...,...,...
125,Thorns Ring,Crafting with Combat Level 7,"{'Bone Fragment': 50, 'Stone': 50, 'Gold Bar': 1}"
126,Glowstone Ring,Crafting with Mining Level 4Stardew Valley Fai...,"{'Solar Essence': 5, 'Iron Bar': 5}"
127,Heavy Tapper,Qi's Secret Walnut Room for 20,"{'Hardwood': 30, 'Radioactive Bar': 1}"
128,Hopper,Qi's Walnut Room,"{'Hardwood': 10, 'Iridium Bar': 1, 'Radioactiv..."


In [79]:
all_ingredients = {}
def helper_ingredients(row):
    ingredients = row['Ingredients']
    for i in ingredients:
        if i in all_ingredients:
            all_ingredients[i] = all_ingredients[i] + ingredients[i]
        else:
            all_ingredients[i] = ingredients[i]
            
crafting_df.apply(helper_ingredients, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
125    None
126    None
127    None
128    None
129    None
Length: 130, dtype: object

In [81]:
all_ingredients

{'Wood': 778,
 'Stone': 760,
 'Iron Bar': 47,
 'Hardwood': 208,
 'Fiber': 279,
 'Sap': 81,
 'Coal': 97,
 'Iridium Ore': 5,
 'Maple Syrup': 1,
 'Copper Bar': 13,
 'Oak Resin': 2,
 'Copper Ore': 28,
 'Clay': 9,
 'Refined Quartz': 25,
 'Earth Crystal': 1,
 'Gold Bar': 30,
 'Pine Tar': 7,
 'Slime': 220,
 'Bat Wing': 40,
 'Iridium Bar': 17,
 'Fire Quartz': 1,
 'Battery Pack': 7,
 'Clam': 1,
 'Coral': 3,
 'Radioactive Ore': 2,
 'Bone Fragment': 128,
 'Solar Essence': 83,
 'Iron Ore': 9,
 'Gold Ore': 9,
 'Void Essence': 51,
 'Ancient Seed': 1,
 'Wild Horseradish': 1,
 'Daffodil': 1,
 'Leek': 1,
 'Dandelion': 1,
 'Spice Berry': 1,
 'Grape': 1,
 'Sweet Pea': 1,
 'Blackberry': 1,
 'Common Mushroom': 1,
 'Hazelnut': 1,
 'Wild Plum': 1,
 'Crocus': 1,
 'Crystal Fruit': 1,
 'Snow Yam': 1,
 'Winter Root': 1,
 'Mixed Seeds': 1,
 'Honey': 1,
 'Coconut': 1,
 'Dragon Tooth': 1,
 'Ginger': 1,
 'Truffle Oil': 1,
 'Acorn': 1,
 'Maple Seed': 1,
 'Pine Cone': 1,
 'Pumpkin': 1,
 'Torch': 1,
 'Bug Meat': 44,
 '